# Basic Classification Example with TensorFlow

This notebook is a companion of [A Visual and Interactive Guide to the Basics of Neural Networks](https://jalammar.github.io/visual-interactive-guide-basics-neural-networks/).

This is an example of how to do classification on a simple dataset in TensorFlow. Basically, we're building a model to help a friend choose a house to buy. She has given us the table below of houses and whether she likes them or not. We're to build a model that takes a house area and number of bathrooms as input, and outputs a prediction of whether she would like the house or not.

| Area (sq ft) (x1) | Bathrooms (x2) | Label (y) |
 | --- | --- | --- |
 | 2,104 |  3 | Good |
 | 1,600 |  3 | Good |
 | 2,400 |  3 | Good |
 | 1,416 | 	2 | Bad |
 | 3,000 | 	4 | Bad |
 | 1,985 | 	4 | Good |
 | 1,534 | 	3 | Bad |
 | 1,427 | 	3 | Good |
 | 1,380 | 	3 | Good |
 | 1,494 | 	3 | Good |
 
 
 


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

C:\Users\theda\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Below, we load the data into a dataframe from Pandas (like excel)

In [2]:
dataframe = pd.read_csv("data.csv")
dataframe = dataframe.drop(["index", "price", "sq_price"], axis=1)
dataframe = dataframe[0:10]
dataframe.head()

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0


These are the input features. Now, we can add the labels (the houses being either good/bad)

In [3]:
dataframe.loc[:, ("y1")] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1] # List from above
dataframe.loc[:, ("y2")] = dataframe["y1"] == 0           # y2 is the negation of y1
dataframe.loc[:, ("y2")] = dataframe["y2"].astype(int)    # Turn TRUE/FALSE values into 1/0
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


This command below reshapes our data into matrices so that TensorFlow can use it.

In [4]:
inputX = dataframe.loc[:, ['area', 'bathrooms']].values
inputY = dataframe.loc[:, ["y1", "y2"]].values

So now our input matrix looks like this:

In [5]:
inputX

array([[2.104e+03, 3.000e+00],
       [1.600e+03, 3.000e+00],
       [2.400e+03, 3.000e+00],
       [1.416e+03, 2.000e+00],
       [3.000e+03, 4.000e+00],
       [1.985e+03, 4.000e+00],
       [1.534e+03, 3.000e+00],
       [1.427e+03, 3.000e+00],
       [1.380e+03, 3.000e+00],
       [1.494e+03, 3.000e+00]])

And this is our "good/bad" matrix

In [6]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=int64)

Parameters are shown below. The learning rate is multiplied by the gradient to determine the next point. The smaller the learning rate, the longer it takes. Training epochs are how many times the data goes through the neural network.

In [7]:
# Parameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

Below we define the tensorflow operations.

In [8]:
x = tf.placeholder(tf.float32, [None, 2])   
#Sets a placeholder to alert TensorFlow that we will be feeding it an array of two float values
            
W = tf.Variable(tf.zeros([2, 2]))           
# Maintain a 2 x 2 float matrix for the weights, start as 0
    
b = tf.Variable(tf.zeros([2]))
#Also maintain two bias values, starting at zero

y_values = tf.add(tf.matmul(x, W), b)
#Multiplies weights by input and adds bias (y = mx + b)
    
y = tf.nn.softmax(y_values)
#softmax is activation function that translates into probability form
    
y_ = tf.placeholder(tf.float32, [None,2])   # For training purposes, we'll also feed you a matrix of labels

Cost function and gradient descent are specified below.

In [9]:
# Cost function: Mean squared error
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)
# Gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [10]:
# Initialize variabls and tensorflow session
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Training shown below.

In [15]:
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Take a gradient descent step using our inputs and labels

    # Display logs per epoch step
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print("Training step", '%04d' % (i), "cost=", "{:.9f}".format(cc)) #, \"W=", sess.run(W), "b=", sess.run(b)

print("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')


Training step 0000 cost= 0.114958666
Training step 0050 cost= 0.109539941
Training step 0100 cost= 0.109539866
Training step 0150 cost= 0.109539807
Training step 0200 cost= 0.109539740
Training step 0250 cost= 0.109539680
Training step 0300 cost= 0.109539606
Training step 0350 cost= 0.109539531
Training step 0400 cost= 0.109539464
Training step 0450 cost= 0.109539405
Training step 0500 cost= 0.109539323
Training step 0550 cost= 0.109539248
Training step 0600 cost= 0.109539203
Training step 0650 cost= 0.109539129
Training step 0700 cost= 0.109539054
Training step 0750 cost= 0.109538987
Training step 0800 cost= 0.109538913
Training step 0850 cost= 0.109538853
Training step 0900 cost= 0.109538786
Training step 0950 cost= 0.109538712
Training step 1000 cost= 0.109538652
Training step 1050 cost= 0.109538577
Training step 1100 cost= 0.109538510
Training step 1150 cost= 0.109538436
Training step 1200 cost= 0.109538369
Training step 1250 cost= 0.109538309
Training step 1300 cost= 0.109538235
T

Now the training is done. We can use our model to run the input data.

In [16]:
sess.run(y, feed_dict={x: inputX })

array([[0.7112522 , 0.2887478 ],
       [0.66498977, 0.33501023],
       [0.73657656, 0.26342347],
       [0.6471879 , 0.3528121 ],
       [0.78335613, 0.2166439 ],
       [0.70069474, 0.29930523],
       [0.6586633 , 0.34133676],
       [0.6482863 , 0.35171372],
       [0.6436828 , 0.35631716],
       [0.65480113, 0.3451989 ]], dtype=float32)

So It's guessing they're all good houses. That makes it get 7/10 correct. A model with a hidden layer should do better.

Thats it.